# Эксперименты с MST

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import networkx as nx
import matplotlib.pyplot as plt
import k_scripts.finance_scripts as fs
import k_scripts.network_scripts as ns
import k_scripts.sampler_scripts as ss
import k_scripts.measures_scripts as ms
import k_scripts.experiments_script as es
import seaborn as sn
from tqdm import tqdm
import importlib
import json 

importlib.reload(ns)
importlib.reload(es)


importlib.reload(ms)

<module 'k_scripts.measures_scripts' from '/home/friday/_homescript/measures-of-uncertainty/k_scripts/measures_scripts.py'>

In [2]:
importlib.reload(ss)

<module 'k_scripts.sampler_scripts' from '/home/friday/_homescript/measures-of-uncertainty/k_scripts/sampler_scripts.py'>

In [3]:
path = './data/DE/DE_17-18_close.csv'
dClose = pd.read_csv(path,index_col='Date')
R = fs.get_daily_returns(dClose)
R = R.drop(['LIN.DE'],axis=1)

In [4]:
R.head(3)

,AAD.DE,ADS.DE,ADV.DE,AFX.DE,AIR.DE,ALV.DE,ARL.DE,BAS.DE,BAYN.DE,BC8.DE,...,SIE.DE,SIX2.DE,SKB.DE,SPR.DE,SZG.DE,SZU.DE,TEG.DE,TKA.DE,TTK.DE,UTDI.DE
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,-0.005579,-0.013642,-0.008337,-0.008979,0.011232,0.014203,0.026694,-0.011338,0.005474,-0.022032,...,-0.014948,-0.004259,0.015531,-0.003213,0.007640,0.012044,-0.010776,0.002828,-0.005836,-0.002527
2017-01-04,-0.001229,-0.019280,-0.010147,-0.008207,-0.002205,0.005313,0.004313,0.005118,0.012821,-0.017237,...,-0.004312,-0.010334,-0.018550,-0.001181,0.000732,-0.003052,-0.000803,-0.015541,-0.004694,-0.008694
2017-01-05,0.006536,-0.014448,-0.010000,-0.003701,0.017968,0.001869,-0.005935,-0.004092,0.005858,-0.002902,...,-0.006503,0.004498,0.001278,0.005569,-0.002490,0.014953,0.007602,0.003743,0.037627,0.010690


In [5]:
Corr = R.corr()
Cov = R.cov()

In [24]:
Cov.shape

(87, 87)

## Нормальное распределение

In [37]:
sampler = np.random.multivariate_normal
reference_network = ns.create_network(R.corr().values)
mean = [0] * Cov.shape[0]


In [44]:
E_measures = dict()
N_min=200
N_max=400
step = 100

for N in tqdm(range(N_min,N_max,step)):
    E_measures[int(N)] = ms.E_measure_MST(reference_network, sampler, sampler_params=[mean, Cov, N])
    
file =  f'./experiments/t_mst_DE/mst_normal_{N_min}-{N_max}.json'
with open(file, 'w') as f:
    json.dump(E_measures, f) 

100%|██████████| 2/2 [01:28<00:00, 44.37s/it]


## Распределение Стьюдента

In [46]:
sampler = ss.multivariate_t
reference_network = ns.create_network(R.corr().values)
mean = [0] * Cov.shape[0]
df=3

In [48]:
E_measures = dict()
N_min=200
N_max=400
step = 100

for N in tqdm(range(N_min,N_max,step)):
    E_measures[int(N)] = ms.E_measure_MST(reference_network, sampler, sampler_params=[mean, Cov, df, N])
    
file =  f'./experiments/t_mst_DE/mst_t_{df}_{N_min}-{N_max}.json'
with open(file, 'w') as f:
    json.dump(E_measures, f) 

100%|██████████| 2/2 [01:46<00:00, 53.02s/it]


## Смешанное распределение

In [6]:
sampler = ss.mixed_t_normal
reference_network = ns.create_network(R.corr().values)
mean = [0] * Cov.shape[0]
df=3
rat = 0.5

In [7]:
E_measures = dict()
N_min=200
N_max=400
step = 100

for N in tqdm(range(N_min,N_max,step)):
    E_measures[int(N)] = ms.E_measure_MST(reference_network, sampler, sampler_params=[mean, Cov, df, N,rat])
    
file =  f'./experiments/mixed_mst_DE/mst_mixed_{df}_{rat}_{N_min}-{N_max}.json'
with open(file, 'w') as f:
    json.dump(E_measures, f) 

100%|██████████| 2/2 [01:28<00:00, 44.50s/it]
